In [1]:
import psycopg2

In [21]:
import pandas as pd
import os
from time import time
from tqdm import tqdm
import psycopg2
from langchain_openai import OpenAIEmbeddings

# OpenAI API Key 설정
os.environ['OPENAI_API_KEY'] = open('API_KEY', 'r').read()
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 데이터 로드 및 전처리
imdb = pd.read_csv('movie_metadata.csv')
imdb = imdb.dropna()
texts = imdb['plot_keywords'][:10].tolist()

# 임베딩 생성 및 저장
embeddings_list = []
for text in tqdm(texts):
    embeddings_list.append(embeddings.embed_query(text))

# 검색할 문자열
test_words = "cyberpunk|dystopia|paraplegic|marine|warrior"  # 검색할 문자열
test_vec = embeddings.embed_query(test_words)
emb_dim = len(test_vec)
k = 5  # 상위 5개 유사 벡터 검색

# 결과 조회용 함수
def show(rows):
    for row in rows:
        print(row)

# 임베딩 파일을 임시 CSV로 저장
import csv
filename = "imdb_embedding.csv"
with open(filename, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["keywords", "embedding"])
    for i in range(len(embeddings_list)):
        writer.writerow([texts[i], embeddings_list[i]])


100%|██████████| 10/10 [00:04<00:00,  2.33it/s]


In [ ]:
# PostgreSQL 연결
conn = psycopg2.connect(host='192.168.0.47', dbname='postgres', user='postgres', password='postgres1016', port=55432)
cursor = conn.cursor()

# 테이블 생성
cursor.execute("DROP TABLE IF EXISTS imdb2;")
cursor.execute("CREATE TABLE IF NOT EXISTS imdb2 (id serial primary key, keywords text, embedding vector({}));".format(len(embeddings_list[0])))
conn.commit()

# 커밋 및 연결 종료
conn.close()

print("Data successfully loaded into PostgreSQL using COPY.")

In [26]:
# shell에서 psql 실행하여 \COPY 명령 실행
import subprocess

# 환경 변수에 패스워드 설정
os.environ["PGPASSWORD"] = "postgres1016" 

# 업로드할 파일 위치
csv_path = os.path.join(os.getcwd(), "imdb_embedding.csv") 

# COPY 명령어를 사용하여 데이터 로드
copy_command = (
    f"psql -h 192.168.0.47 -p 55432 -d postgres -U postgres -c "
    f"\"\\COPY imdb2 (keywords, embedding) FROM '{csv_path}' WITH (FORMAT CSV, HEADER TRUE);\""
)
subprocess.run(copy_command, shell=True, check=True)

print("Data successfully loaded into PostgreSQL using \\COPY.")

COPY 10
Data successfully loaded into PostgreSQL using \COPY.


In [28]:
# PG에 연결
conn = psycopg2.connect(host='192.168.0.47', dbname='postgres', user='postgres', password='postgres1016', port=55432)
cursor = conn.cursor()

# 유사도 기반 검색
# <-> : L2거리
s = time()
cursor.execute("""
    SELECT id,
        keywords,
        embedding <-> '{}' AS similarity
    FROM imdb2
    ORDER BY similarity
    LIMIT 5
""".format(test_vec))

time_spent = time() - s
print("time:", time_spent, "\n")

result = cursor.fetchall()
show(result)

# PG에 커밋
conn.commit()
conn.close()

time: 0.014770030975341797 

(1, 'avatar|future|marine|native|paraplegic', 0.7837149072848963)
(3, 'bomb|espionage|sequel|spy|terrorist', 1.065572671360541)
(5, 'alien|american civil war|male nipple|mars|princess', 1.073223860135542)
(4, 'deception|imprisonment|lawlessness|police officer|terrorist plot', 1.111155303764997)
(8, 'artificial intelligence|based on comic book|captain america|marvel cinematic universe|superhero', 1.114257170587567)
